# Todos

## General

- bale ta ballast trips apo ships tensor pisw sto ships df ❌
- ananewne ka8w fora ta contracts_df kai ships_df me oti allagh ginetai stous contracts_tensor kai ships_tensor


## Step

### Allakse to state otan allazei h mera ❌

afou looparw se ola ta available ships shmainei oti h mera allazei

   #### - diamorfwse katallhla to ships log ❌
   
          * to ships log prepei na exei values >= 0 ❌
          * apo mh mhdenika ships log values afairese 1 epeidh perase mia mera ❌
          * an prokypsoun ships_log[ship] = 0 tote kane to `ship_availability` autou tou ship 1 ❌
          * osa ships einai available (exoun ships_log[ship] == 0) bainoun se lista `available_ships` pou 8a thn allazeis ka8e mera. ❌
           

   #### -  kane generate 4 nea contracts ❌

### Pws ypologizw to `reward` apo to action  ✅
    

   #### - Value of contract ✅
   #### - CII ✅
   #### - Lateness ✅
   #### - Ftiakse sugentrwtiko `calculate_reward(selected_ship_idx,selected_contract,selected_speed)` function ✅

### Pws na kanw swsta update to `state`  ✅
  

      
   #### - contract tensor ✅
   #### - ship tensor ✅
   #### - contract mask ✅
   #### - ship mask ✅
   #### - ship_log ✅
        

## Train 

ti einai ta `advantages`, `returns` kai giati xrhsimopoioyntai:
        
- ❌ sthn baseline_net.update(observations, returns) gia to update to baselineNet
- ❌ sthn PolicyGradient.update_policy(observations, actions, advantages)gia to update ths policygradient 

# Code

In [1]:
# autoreloading modules that i change in vscode
%load_ext autoreload
%autoreload 2

In [2]:
import gym
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [ ]:
# run this cell when needing to reload the classes CarbonEnv, BaselineNet, PolicyNet

del sys.modules['models.models']
del sys.modules['env.env']
del sys.modules['training.training_functions']
del sys.modules['utils.utils']

from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel
from training.training_functions import PolicyGradient

In [3]:
from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel
from training.training_functions import PolicyGradient

In [4]:
envo = CarbonEnv()

2021-11-19 13:00:59.986337: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-19 13:00:59.987336: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
model = PolicyGradient(envo)

In [6]:
year_data, num_steps = model.train()

Ksekina to year: 0
Xronia: 1 kai mera: 0
Ta available ships einai [1, 2, 3, 4]
Den phra contract gia to available ship 1 
Den phra contract gia to available ship 2 
Den phra contract gia to available ship 3 
Den phra contract gia to available ship 4 
Xronia: 1 kai mera: 1
Ta available ships einai [1, 2, 3, 4]
Den phra contract gia to available ship 1 
Den phra contract gia to available ship 2 
Den phra contract gia to available ship 3 
Den phra contract gia to available ship 4 
Xronia: 1 kai mera: 2
Ta available ships einai [1, 2, 3, 4]
Den phra contract gia to available ship 1 
Den phra contract gia to available ship 2 
Den phra contract gia to available ship 3 
Den phra contract gia to available ship 4 
Xronia: 1 kai mera: 3
Ta available ships einai [1, 2, 3, 4]
Den phra contract gia to available ship 1 
Den phra contract gia to available ship 2 
Den phra contract gia to available ship 3 
Den phra contract gia to available ship 4 
Xronia: 1 kai mera: 4
Ta available ships einai [1, 2,

In [7]:
baseline_net = BaselineNet(128,1)
policy_net = PolicyNet(128,13)

In [8]:
rewards_array_current_year = year_data["reward"]
states_array_current_year = year_data["states"]
actions_array_current_year = year_data["action"]

In [9]:
def get_returns(rewards_array):
    T = len(rewards_array)
    discounts = np.logspace(0, T, num=T, base=0.99, endpoint=False)
    returns = np.array([np.sum(discounts[: T - t] * rewards_array[t:]) for t in range(T)])
    return returns

In [10]:
def get_advantage(returns, states):
    value_array = np.array([])
    for state in states:
        value = baseline_net.forward(state).numpy()
        value_array = np.append(value_array,value)
    advantages = returns - value_array
    advantages = (advantages - np.mean(advantages)) / np.sqrt(np.sum(advantages ** 2))
    return advantages

In [11]:
returns_array_current_year = get_returns(rewards_array_current_year)

advantages_array_current_year = get_advantage(
    returns_array_current_year, states_array_current_year)


In [ ]:
# # expand dims gia ta gradients
# actions_array_current_year = tf.convert_to_tensor(actions_array_current_year)
# actions_array_current_year = tf.expand_dims(actions_array_current_year, axis=0)
# actions_array_current_year.shape

In [ ]:
# returns_array_current_year = tf.convert_to_tensor(returns_array_current_year)
# returns_array_current_year = tf.expand_dims(returns_array_current_year, axis=0)
# returns_array_current_year.shape

In [ ]:
# advantages_array_current_year = tf.convert_to_tensor(advantages_array_current_year)
# advantages_array_current_year = tf.expand_dims(advantages_array_current_year, axis=0)
# advantages_array_current_year.shape

In [ ]:
baseline_net.update(states_array_current_year,returns_array_current_year)

In [ ]:
min_log_prob = tf.reduce_min(
    tf.boolean_mask(log_probs, tf.math.is_finite(log_probs)), keepdims=True
)

In [ ]:
min_log_prob

In [ ]:
log_probs_no_inf = tf.where(tf.math.is_inf(log_probs), 1000 * min_log_prob, log_probs)
log_probs_no_inf

In [ ]:
a = tf.math.multiply(log_probs_no_inf, tf.cast(advantages_array_current_year, tf.float32))
a

In [ ]:
log_probs_no_inf * advantages_array_current_year

In [ ]:
entropy_loss_weight = 0.0001

In [ ]:
b = entropies * entropy_loss_weight
b

In [ ]:
loss= -tf.math.reduce_mean((a+b),keepdims=True)
loss

In [ ]:
tf.repeat(loss,13)

In [ ]:
[
                    policy_net.action_distribution(state)[1].log_prob(action)
                    for state, action in zip(states_array_current_year, actions_array_current_year)
                ],

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>]

In [ ]:
def update_policy(states_array, one_action, one_advantage):
    #one_state is already a tensor
    one_action = tf.convert_to_tensor(one_action)
    one_advantage = tf.convert_to_tensor(one_advantage)
    with tf.GradientTape() as tape:
        log_prob = policy_net.action_distribution(one_state)[1].log_prob(one_action)
        loss = -tf.math.reduce_mean(log_prob * tf.cast(one_advantage, tf.float32))
    grads = tape.gradient(loss, policy_net.policy_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, policy_net.policy_model.trainable_weights))

In [ ]:
l = [1,1,1]

In [ ]:
np.mean(l)

In [ ]:
each_year_actions_list = np.load('results_all/results_3/actions.npy',allow_pickle=True)

In [ ]:
each_year_baseline_loss_list = np.load('results_all/results_3/baseline_loss.npy',allow_pickle=True)

In [ ]:
each_year_policynet_loss_list = np.load('results_all/results_3/policynet_loss.npy',allow_pickle=True)

In [ ]:
each_year_avg_reward_list = np.load('results_all/results_3/avg_rewards.npy',allow_pickle=True)

In [ ]:
each_year_reward_list = np.load('results_all/results_3/rewards.npy',allow_pickle=True)

In [ ]:
print(each_year_actions_list[0])

### Tensorflow Tip

Assign values to a tensor at specific indices

In [ ]:
input_array = np.array([2, 4, 7, 11, 3, 8, 9, 19, 11, 7])
inplace_array = np.array([10, 20])
indices_array = np.array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

# [[2], [6]] 
indices = tf.cast(tf.where(tf.equal(indices_array,1)),tf.int32)
print(indices)

# [0, 0, 10, 0, 0, 0, 20, 0, 0, 0]
scatter = tf.scatter_nd(indices, inplace_array, shape=tf.shape(input_array))
print(scatter)

# [1, 1, 0, 1, 1, 1, 0, 1, 1, 1]
inverse_mask = tf.cast(tf.math.logical_not(indices_array), tf.int32)
print(inverse_mask)

# [2, 4, 0, 11, 3, 8, 0, 19, 11, 7]
input_array_zero_out = tf.multiply(inverse_mask, input_array)
print(input_array_zero_out)

# [2, 4, 10, 11, 3, 8, 20, 19, 11, 7]
output = tf.add(input_array_zero_out, tf.cast(scatter, tf.int32))
print(output)

In [ ]:
# [[2], [6]] 
indices = tf.cast(tf.where(tf.equal(indices_array,1)),tf.int32)
print(indices)

In [ ]:
# [0, 0, 10, 0, 0, 0, 20, 0, 0, 0]
scatter = tf.scatter_nd(indices, inplace_array, shape=tf.shape(input_array))
print(scatter)

In [ ]:
# [1, 1, 0, 1, 1, 1, 0, 1, 1, 1]
inverse_mask = tf.cast(tf.math.logical_not(indices_array), tf.int32)
print(inverse_mask)

In [ ]:
# [2, 4, 0, 11, 3, 8, 0, 19, 11, 7]
input_array_zero_out = tf.multiply(inverse_mask, input_array)
print(input_array_zero_out)

In [ ]:
# [2, 4, 10, 11, 3, 8, 20, 19, 11, 7]
output = tf.add(input_array_zero_out, tf.cast(scatter, tf.int32))
print(output)

In [ ]:
# encoder mlp

# see https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
# see https://www.tensorflow.org/tutorials/generative/autoencoder
# Create an Integer Categorical Feature for contract_type